In [3]:
import pandas as pd
from os import path
import json

In [4]:
DATA_DIR = "../../data"

In [5]:
seeds = [4, 18, 27, 36, 42]

In [6]:
split_results = {}
for seed in seeds:
    results_a = pd.read_csv(path.join(
        DATA_DIR, 
        "evaluations/protein_bert", 
        f"{str(seed)}a", 
        "all.csv"
    ), sep="\t", header=None)
    results_b = pd.read_csv(path.join(
        DATA_DIR, 
        "evaluations/protein_bert", 
        f"{str(seed)}b", 
        "all.csv"
    ), sep="\t", header=None)
    results_a.columns = ["model_name", "patience_stop", "patience_lr", "lr", "F1", "MCC", "Acc", "Precision", "Recall", "AUC"]
    results_b.columns = ["model_name", "patience_stop", "patience_lr", "lr", "F1", "MCC", "Acc", "Precision", "Recall", "AUC"]
    
    split_results[str(seed) + "a"] = results_a    
    split_results[str(seed) + "b"] = results_b

FileNotFoundError: [Errno 2] No such file or directory: '../../data/evaluations/protein_bert/4a/all.csv'

In [8]:
df_list = [value for key, value in split_results.items()]
# Concatenate results for all data splits
df_concat = pd.concat(df_list)

In [9]:
df_concat

,model_name,patience_stop,patience_lr,lr,F1,MCC,Acc,Precision,Recall,AUC
0,protein_bert,3,3,0.00010,0.215909,0.276221,0.788991,0.863636,0.123377,0.558688
1,protein_bert,3,3,0.00050,0.290155,0.286353,0.790520,0.717949,0.181818,0.579909
2,protein_bert,3,3,0.00001,0.174419,0.237050,0.782875,0.833333,0.097403,0.545701
3,protein_bert,3,3,0.00005,0.000000,0.000000,0.764526,0.000000,0.000000,0.500000
4,protein_bert,4,3,0.00050,0.365297,0.297449,0.787462,0.615385,0.259740,0.604870
...,...,...,...,...,...,...,...,...,...,...
11,protein_bert,6,3,0.00050,0.166667,0.198954,0.778831,0.700000,0.094595,0.541112
12,protein_bert,8,4,0.00001,0.282927,0.204364,0.767773,0.508772,0.195946,0.569107
13,protein_bert,8,4,0.00005,0.291457,0.234173,0.777251,0.568627,0.195946,0.575293
14,protein_bert,8,4,0.00010,0.308411,0.214575,0.766193,0.500000,0.222973,0.577466


In [12]:
grouped = df_concat.groupby(["patience_stop", "patience_lr", "lr"])
df_means = grouped.mean()

In [13]:
# Means across the different data splits
df_means

F1       MCC       Acc  Precision  \
patience_stop patience_lr lr                                                 
3             3           0.00001  0.182116  0.184428  0.805466   0.513123   
                          0.00005  0.223622  0.197344  0.803498   0.490765   
                          0.00010  0.262153  0.236381  0.806574   0.582830   
                          0.00050  0.216287  0.192138  0.801263   0.513189   
4             3           0.00001  0.238320  0.238188  0.808160   0.641785   
                          0.00005  0.245135  0.225618  0.806398   0.573904   
                          0.00010  0.280859  0.239896  0.804375   0.557861   
                          0.00050  0.300178  0.241629  0.799907   0.534031   
6             3           0.00001  0.229234  0.226182  0.807030   0.624529   
                          0.00005  0.282720  0.246470  0.805293   0.581046   
                          0.00010  0.304934  0.265914  0.808064   0.593477   
                          0.00050  0.318652  0.260526  0.802933   0.554318   
8             4           0.00001  0.266007  0.246692  0.807818   0.613166   
                          0.00005  0.328905  0.276909  0.809786   0.577567   
                          0.00010  0.326120  0.272119  0.805423   0.572848   
                          0.00050  0.288155  0.250027  0.801481   0.580269   

                                     Recall       AUC  
patience_stop patience_lr lr                           
3             3           0.00001  0.113274  0.547804  
                          0.00005  0.150017  0.560568  
                          0.00010  0.178601  0.572743  
                          0.00050  0.144625  0.556851  
4             3           0.00001  0.149428  0.563107  
                          0.00005  0.162059  0.566994  
                          0.00010  0.195304  0.577622  
                          0.00050  0.220135  0.584203  
6             3           0.00001  0.146917  0.561536  
                          0.00005  0.191010  0.576553  
                          0.00010  0.211391  0.586241  
                          0.00050  0.233752  0.591721  
8             4           0.00001  0.173537  0.571465  
                          0.00005  0.233730  0.595088  
                          0.00010  0.238479  0.594946  
                          0.00050  0.208441  0.581808

In [16]:
df_means.round(3).to_csv(path.join(DATA_DIR, "evaluations/protein_bert/5x2cv_means.csv"))

In [17]:
df_means.round(3).to_csv(path.join(DATA_DIR, "evaluations/protein_bert/5x2cv_means_excel.csv"), sep="\t")

In [1]:
from statistics import mean
from sklearn.metrics import accuracy_score
import numpy as np
import csv
import scipy.stats
import itertools

In [2]:
test_data = pd.read_csv(path.join(DATA_DIR, "chen/deduplicated/crossval/chen_4_b.csv"), index_col=0).sort_index()
test_data.head()

NameError: name 'pd' is not defined

In [13]:
sapiens_missing_indices = [1237, 1236, 1631, 1356, 1354, 1355, 1749, 2216, 1752]
sapiens_missing = test_data.sort_index().reset_index()[test_data.sort_index().reset_index()["index"].isin(sapiens_missing_indices)].index

In [14]:
onehot_missing_indices = [1921]
onehot_missing = test_data.sort_index().reset_index()[test_data.sort_index().reset_index()["index"] == 1921].index

In [15]:
y_preds = []
i = 0
for model_name in ["kNN", "logistic_regression", "random_forest", "multilayer_perceptron", "SVM", "gradient_boosting"]:
    for data_rep in ["integer_encoded", "pybiomed", "protparam", "bert", "seqvec", "sapiens", "onehot"]:
        for prepro in ["scaling"]:
            #print(f"{i}: {model_name}, {data_rep}")
            i += 1
            with open(path.join(DATA_DIR, f"evaluations/5x2cv/training_split_4_a/{model_name}_{data_rep}_{prepro}.csv"), "r") as f:
                reader = csv.reader(f)
                row = [int(num) for num in next(reader)]
                if data_rep == "sapiens":  # TODO !!!
                    for idx in sapiens_missing:
                        row.insert(idx, 0)
                if data_rep == "onehot":
                    row.insert(507, 0)
                y_preds.append(np.array(row))

In [16]:
print(len(y_preds))

42


In [17]:
y_true = np.array(test_data["Y"])

In [18]:
# Looney 1988
# https://www.sciencedirect.com/science/article/pii/0167865588900165
def ftest_compact(y_test, test_preds):
    n = len(y_test)
    accuracies = [accuracy_score(y_test, pred) for pred in test_preds]
    avg_acc = mean(accuracies)
    count = len(test_preds)
    m = []
    ssa = (n * sum(acc**2 for acc in accuracies)) - (n * count * avg_acc**2)
    
    for j in range(n):
        mj = sum([1 for preds in test_preds if preds[j] == y_test[j]])
        m.append(mj)
        
    ssb = (1. / count) * sum(mj ** 2 for mj in m) - (count * n * avg_acc ** 2)
    sst = count * n * avg_acc * (1 - avg_acc)
    ssab = sst - ssa - ssb
    msa = ssa / (count - 1)
    msab = ssab / ((count - 1)*(n - 1))
    f = msa / msab
    
    degrees_of_freedom_1 = count - 1
    degrees_of_freedom_2 = degrees_of_freedom_1 * n

    p_value = scipy.stats.f.sf(f, degrees_of_freedom_1, degrees_of_freedom_2)

    return f, p_value

In [19]:
f, p_value = ftest_compact(y_true, y_preds)

In [20]:
p_value

3.534669655248187e-183

In [90]:
f

24.858923968081413